In [1]:
import os
from dotenv import load_dotenv
import subprocess
from langchain import OpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool

In [2]:
os.environ['OPENAI_API_VERSION'] = '2024-05-01-preview'
load_dotenv()

AZURE_OPENAI_API_KEY = "4a8f724e3ef84157bc5378553d3dec14"
AZURE_OPENAI_ENDPOINT = "https://elevengpt.openai.azure.com/"

model1 = 'gpt4_32k'
model2 = 'eleven_gpt_35_turbo_16k'

In [3]:
llm = AzureChatOpenAI(azure_deployment=model1, api_version='2024-05-01-preview', temperature=0, max_tokens=None, timeout=None, max_retries=2)

tools = load_tools(['llm-math'],llm=llm)

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True
)



c:\Users\MatthieuKAEPPELIN\anaconda3\envs\longchain_training\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
c:\Users\MatthieuKAEPPELIN\anaconda3\envs\longchain_training\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [4]:
zero_shot_agent("Combient fait (4.5*2.1)^2.2 ?")

c:\Users\MatthieuKAEPPELIN\anaconda3\envs\longchain_training\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
This is a mathematical question. I need to calculate the expression (4.5*2.1)^2.2.
Action: Calculator
Action Input: (4.5*2.1)**2.2
Observation: Answer: 139.94261298333066
Thought:I have calculated the expression and have the result.
Final Answer: The result of the expression (4.5*2.1)^2.2 is approximately 139.94.

> Finished chain.


{'input': 'Combient fait (4.5*2.1)^2.2 ?',
 'output': 'The result of the expression (4.5*2.1)^2.2 is approximately 139.94.'}

In [5]:
prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

tools.append(llm_tool)

c:\Users\MatthieuKAEPPELIN\anaconda3\envs\longchain_training\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [6]:
print(tools)

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions

In [7]:
agentzero_shot_agent = initialize_agent(agent="zero-shot-react-description",
                                        tools=tools,
                                        llm=llm,
                                        verbose=True,
                                        max_iterations=3,
                                        handle_parsing_errors=True
                                        )

In [8]:
zero_shot_agent("Quelle est la capitale de la Norvége ?")



> Entering new AgentExecutor chain...
This question is asking for the capital of Norway. I don't need to use the calculator for this, as it's a general knowledge question.
Final Answer: La capitale de la Norvège est Oslo.

> Finished chain.


{'input': 'Quelle est la capitale de la Norvége ?',
 'output': 'La capitale de la Norvège est Oslo.'}